In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import sounddevice as sd
import numpy as np
from pyzbar.pyzbar import decode
import cv2
import os
import re
from datetime import datetime
from IPython.display import display, clear_output
# List untuk menyimpan semua faktur
data_faktur = []
processed_faktur_numbers = set()
scanned_links = []
# Kolom DataFrame
columns = [
    "npwp_penjual", "nama_penjual", "alamat_penjual", "npwp_pembeli", "nama_pembeli",
    "alamat_pembeli", "kode_jenis_transaksi", "jenis_faktur", "nomor_seri_faktur",
    "tanggal_faktur", "jumlah_dpp", "jumlah_ppn", "jumlah_ppnbm", "status_faktur",
    "status_approve", "referensi", "nama_barang", "harga_satuan", "jumlah_barang",
    "harga_total", "diskon", "dpp_satuan", "ppn_satuan", "ppnbm_satuan", "link_faktur",
    "scan_time"
]
def parse_faktur(xml_string, qr_url):
    """Parse XML faktur dari QR Code dan ubah ke dictionary"""
    root = ET.fromstring(xml_string)

    # Ambil data dasar
    faktur_data = {
        "npwp_penjual": root.findtext("npwpPenjual"),
        "nama_penjual": root.findtext("namaPenjual"),
        "alamat_penjual": root.findtext("alamatPenjual"),
        "npwp_pembeli": root.findtext("npwpLawanTransaksi"),
        "nama_pembeli": root.findtext("namaLawanTransaksi"),
        "alamat_pembeli": root.findtext("alamatLawanTransaksi"),
        "kode_jenis_transaksi": root.findtext("kdJenisTransaksi"),
        "jenis_faktur": root.findtext("fgPengganti"),
        "nomor_seri_faktur": root.findtext("nomorFaktur"),
        "tanggal_faktur": root.findtext("tanggalFaktur"),
        "jumlah_dpp": root.findtext("jumlahDpp"),
        "jumlah_ppn": root.findtext("jumlahPpn"),
        "jumlah_ppnbm": root.findtext("jumlahPpnBm"),
        "status_faktur": root.findtext("statusFaktur"),
        "status_approve": root.findtext("statusApproval"),
        "referensi": root.findtext("referensi"),
        "link_faktur": qr_url,
        "scan_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }

    # Periksa jika NPWP Pembeli adalah 000000000000000 (indikasi NIK digunakan)
    npwp_pembeli = faktur_data["npwp_pembeli"]
    nama_pembeli = faktur_data["nama_pembeli"]

    if npwp_pembeli == "000000000000000" and "#NIK#" in nama_pembeli:
        # Gunakan regex untuk mencari angka sebelum '#NIK#'
        match = re.search(r"(\d+)#NIK#", nama_pembeli)
        if match:
            faktur_data["npwp_pembeli"] = match.group(1)  # Gunakan angka sebagai NPWP

    # Ambil detail transaksi
    detail_transaksi = root.findall("detailTransaksi")
    for detail in detail_transaksi:
        detail_data = {
            "nama_barang": detail.findtext("nama"),
            "harga_satuan": detail.findtext("hargaSatuan"),
            "jumlah_barang": detail.findtext("jumlahBarang"),
            "harga_total": detail.findtext("hargaTotal"),
            "diskon": detail.findtext("diskon"),
            "dpp_satuan": detail.findtext("dpp"),
            "ppn_satuan": detail.findtext("ppn"),
            "ppnbm_satuan": detail.findtext("ppnbm"),
        }
        # Gabungkan data faktur dengan detail transaksi
        faktur_data.update(detail_data)
        data_faktur.append(faktur_data.copy())

    return faktur_data

def play_sound():
    """Bunyikan suara penanda bahwa scan berhasil"""
    fs = 44100  # Sample rate
    seconds = 0.5
    t = np.linspace(0, seconds, int(fs * seconds), endpoint=False)
    tone = 0.5 * np.sin(2 * np.pi * 1000 * t)
    sd.play(tone, fs)
    sd.wait()

def scan_qr():
    """Buka kamera dan baca QR Code, lalu proses XML dari URL"""
    cap = cv2.VideoCapture(1)
    print("Arahkan QR Code ke kamera...")

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        decoded_objects = decode(frame)
        for obj in decoded_objects:
            qr_url = obj.data.decode("utf-8")
            print(f"QR Terdeteksi: {qr_url}")

            try:
                response = requests.get(qr_url, timeout=10)
                if response.status_code == 200:
                    faktur_data = parse_faktur(response.text, qr_url)  # Kirim URL ke parse_faktur

                    nomor_faktur = faktur_data["nomor_seri_faktur"]
                    if nomor_faktur in processed_faktur_numbers:
                        print(f"⚠ WARNING: Faktur {nomor_faktur} sudah diproses. Lewati faktur ini.")
                    else:
                        processed_faktur_numbers.add(nomor_faktur)
                        data_faktur.append(faktur_data)
                        scanned_links.append(qr_url)
                        if len(scanned_links) > 3:
                            scanned_links.pop(0)
                        print(f"✅ Faktur {nomor_faktur} berhasil diproses.")

                        play_sound()
                        update_preview()
                else:
                    print("❌ ERROR: Gagal mengunduh XML.")
            except requests.exceptions.Timeout:
                print("⚠ WARNING: Waktu pemrosesan lebih dari 10 detik. Silakan scan ulang QR.")
            except Exception as e:
                print(f"❌ ERROR: Terjadi kesalahan: {e}")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

def update_preview():
    """Update dan tampilkan preview data di Excel"""
    df = pd.DataFrame(data_faktur, columns=columns)
    
    file_path = "faktur_pembelian.xlsx"
    df.to_excel(file_path, index=False)

    preview_data(file_path)

def preview_data(file_path):
    """Preview data dalam file Excel setelah disimpan"""
    if os.path.exists(file_path):
        clear_output(wait=True)
        df_preview = pd.read_excel(file_path)
        print("\n=== 📊 Preview Data dalam Excel ===")
        for link in scanned_links:
            print(link)
        display(df_preview.tail(20))

scan_qr()



=== 📊 Preview Data dalam Excel ===
http://svc.efaktur.pajak.go.id/validasi/faktur/015480486007000/0002421534181/3031300D0609608648016503040201050004200650DA02ECA35A9CE979C672B108646834F15E7A163916720AC81F809F9CB614
http://svc.efaktur.pajak.go.id/validasi/faktur/085235794601000/0112434536447/3031300D0609608648016503040201050004203AC4D7F9CB50BFB7026CACE0CBE5408C53732A5F0E290F65298534348769DD5B
http://svc.efaktur.pajak.go.id/validasi/faktur/025181819506000/0102493510477/3031300D060960864801650304020105000420629B747A3ADDC621D5948FEC7AA21C3BDF4FBFFCE0379B065A5BC4C840DDB511


,npwp_penjual,nama_penjual,alamat_penjual,npwp_pembeli,nama_pembeli,alamat_pembeli,kode_jenis_transaksi,jenis_faktur,nomor_seri_faktur,tanggal_faktur,...,nama_barang,harga_satuan,jumlah_barang,harga_total,diskon,dpp_satuan,ppn_satuan,ppnbm_satuan,link_faktur,scan_time
1300,754890895514000,CV MUARA BERKAT,"JL KI SORENG RT 001 RW 003 , KAB. BLORA",965593155514000,CV JAYA SLAMET,JL RAYA CEPU NO 14 RT 006 RW 004 KEC CEPU,1,0,102450605565,30/12/2024,...,BONNS BITE 2000 40.,54954.9500,20,1.099099e+06,0.0000,1.099099e+06,120900.9000,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:25
1301,754890895514000,CV MUARA BERKAT,"JL KI SORENG RT 001 RW 003 , KAB. BLORA",965593155514000,CV JAYA SLAMET,JL RAYA CEPU NO 14 RT 006 RW 004 KEC CEPU,1,0,102450605565,30/12/2024,...,BIG ARIES 2000 40.,54954.9500,25,1.373874e+06,0.0000,1.373874e+06,151126.1300,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:25
1302,754890895514000,CV MUARA BERKAT,"JL KI SORENG RT 001 RW 003 , KAB. BLORA",965593155514000,CV JAYA SLAMET,JL RAYA CEPU NO 14 RT 006 RW 004 KEC CEPU,1,0,102450605565,30/12/2024,...,BIG ARIES 2000 40.,54954.9500,25,1.373874e+06,0.0000,1.373874e+06,151126.1300,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:25
1303,25181819506000,CV TJAHYONO ABADI,"DS LORAM KULON RT 008 RW 001 , KAB. KUDUS",965593155514000,CV.JAYA SLAMET,Jl Raya Cepu No.14 RT.006 RW.004 Cepu - Cepu,1,0,102493511254,19/12/2024,...,JJ CLOUD9 CRUN CHOCOLATE (10D 24) 500,94600.0000,48,4.540800e+06,136224.0000,4.404576e+06,484503.3600,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:51
1304,25181819506000,CV TJAHYONO ABADI,"DS LORAM KULON RT 008 RW 001 , KAB. KUDUS",965593155514000,CV.JAYA SLAMET,Jl Raya Cepu No.14 RT.006 RW.004 Cepu - Cepu,1,0,102493511254,19/12/2024,...,BNS JJ CLOUD9 CRUN CHOCO (10D 24) 500,0.0000,1,0.000000e+00,0.0000,0.000000e+00,0.0000,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:51
1305,25181819506000,CV TJAHYONO ABADI,"DS LORAM KULON RT 008 RW 001 , KAB. KUDUS",965593155514000,CV.JAYA SLAMET,Jl Raya Cepu No.14 RT.006 RW.004 Cepu - Cepu,1,0,102493511254,19/12/2024,...,JJ CLOUD9 CRUN CHOCO~14 (6D 24) 1000,109152.0000,6,6.549120e+05,19647.3600,6.352646e+05,69879.1100,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:51
1306,25181819506000,CV TJAHYONO ABADI,"DS LORAM KULON RT 008 RW 001 , KAB. KUDUS",965593155514000,CV.JAYA SLAMET,Jl Raya Cepu No.14 RT.006 RW.004 Cepu - Cepu,1,0,102493511254,19/12/2024,...,BNS JJ CLOUD9 CRUN CHOCO~14 (6D 24),0.0000,3,0.000000e+00,0.0000,0.000000e+00,0.0000,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:51
1307,25181819506000,CV TJAHYONO ABADI,"DS LORAM KULON RT 008 RW 001 , KAB. KUDUS",965593155514000,CV.JAYA SLAMET,Jl Raya Cepu No.14 RT.006 RW.004 Cepu - Cepu,1,0,102493511254,19/12/2024,...,BNS JJ CLOUD9 CRUN CHOCO~14 (6D 24),0.0000,3,0.000000e+00,0.0000,0.000000e+00,0.0000,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:21:51
1308,15480486007000,PT. FASTRATA BUANA,"JL. SUCI NO.75, SUSUKAN, CIRACAS, KOTA ADM. JA...",965593155514000,CV. JAYA SLAMET,"JL. RAYA NO. 14 RT.006 RW.004 CEPU CEPU , BLO...",1,0,2421534181,14/12/2024,...,"SP MERAH (RTG, 20 X 10 X 6 GR) (JW/PG)",162162.1600,20,3.243243e+06,71351.3514,3.171892e+06,348908.1100,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:22:03
1309,15480486007000,PT. FASTRATA BUANA,"JL. SUCI NO.75, SUSUKAN, CIRACAS, KOTA ADM. JA...",965593155514000,CV. JAYA SLAMET,"JL. RAYA NO. 14 RT.006 RW.004 CEPU CEPU , BLO...",1,0,2421534181,14/12/2024,...,"SP MIX (RTG,12 X 10 X 23 GR)",180180.1800,10,1.801802e+06,39639.6396,1.762162e+06,193837.8900,0,http://svc.efaktur.pajak.go.id/validasi/faktur...,2025-04-29 11:22:03


KeyboardInterrupt: 